In [1]:
import os
import javalang

PATH = './data/other_java/'
#PATH = './data/ttt/'
#'ast_save'

from gensim.models.word2vec import Word2Vec
model= Word2Vec(min_count=4, size=200, workers=6, max_final_vocab=1000000)
#w2v = Word2Vec(corpus, size=size, workers=16, sg=1, max_final_vocab=3000)
#w2v.save(data_path+'train/embedding/node_w2v_' + str(size))


def get_ast(filename):
    assert os.path.isfile(filename)
    try:
        with open(filename, 'r', encoding="utf-8") as f:
            code = f.read()
    except:
        return None
    try:
        ast = javalang.parse.parse(code)
        #seq = trans2tokenseq(ast)
        return ast
    except:
        try:
            tokens = javalang.tokenizer.tokenize(code)
            parser = javalang.parser.Parser(tokens)
            ast = parser.parse_member_declaration()
            return ast
        except:
            return None
        
'''
def get_ast(filename):
    assert not os.path.isdir(filename)
    with open(filename, 'r', encoding="utf-8") as f:
        code = f.read()
    try:
        ast = javalang.parse.parse(code)
        #seq = trans2tokenseq(ast)
        return ast
    except:
        try:
            tokens = javalang.tokenizer.tokenize(code)
            parser = javalang.parser.Parser(tokens)
            ast = parser.parse_member_declaration()
            return ast
        except:
            return None
'''    

def get_functions(ast):
    return list(ast.filter(javalang.tree.MethodDeclaration))

def get_function_name(function_ast):
    return function_ast.name
    
from utils import get_sequence

def trans_to_sequences(ast):
    sequence = []
    get_sequence(ast, sequence)
    return sequence

def get_sentences(path):
    #sentences = []
    files = os.listdir(path) # 得到文件夹下的所有文件名称
    for file in files:
        if file[0] == '.':  # 隐藏文件或文件夹跳过
            continue
        if os.path.isdir(path+file):
            yield from get_sentences(path+file+'/')
        elif file[-5:].lower()=='.java':
            file_path = path+file
            ast = get_ast(file_path)
            if ast == None:
                continue
            functions = get_functions(ast)
            for _, node in functions:
                #yield get_function_name(node)
                yield trans_to_sequences(node)
                #sentences.append(trans_to_sequences(node))
    #yield None
    #yield []


In [2]:
model.build_vocab(get_sentences(PATH))

In [3]:
word2vec = model.wv

vocab = word2vec.vocab
max_token = word2vec.vectors.shape[0]
dim = word2vec.vectors.shape[1]

print(max_token)  # 字典长度
print(model.corpus_count)  # 句子总数
print(dim)

981088
3471317
200


In [4]:
model.train(get_sentences(PATH), total_examples=model.corpus_count, epochs=model.epochs)
model.save('./word2vec_Model/word2vec')

In [6]:
word2vec = model.wv

vocab = word2vec.vocab
max_token = word2vec.vectors.shape[0]
dim = word2vec.vectors.shape[1]

print(max_token)  # 字典长度
print(model.corpus_count)  # 句子总数
print(dim)

981088
3471317
200
